In [15]:
# tutorial: https://github.com/ashima0109/VGG-classification

In [16]:
def imports():
    '''
    imports(): All of the necessary imports to run the code. Users must have
    the most up-to-date versions of the packages/libraries in order to
    successfully run the code.
    '''

from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras.callbacks import TensorBoard
from keras import models
import tensorboard
from sklearn.model_selection import train_test_split
import random
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
import time
import gradio as gr
import sys

In [58]:
def build(width, height, depth):
    '''
    build(): Constructs the VGG model.
    \t:param width: width of input images
    \t:param height: height of input images
    \t:param depth: depth (i.e., number of color channels) of input images
    \t:return: constructed model
    '''

    # initialize model, input shape, and channel dimension
    model = Sequential()
    inputShape = (height, width, depth)
    chanDim = -1  

    # CONV -> RELU -> POOL layer set
    model.add(Conv2D(32, (3, 3), padding = "same", input_shape = inputShape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis = chanDim))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.4))

    # (CONV -> RELU) * 2 -> POOL layer set
    model.add(Conv2D(64, (3, 3), padding = "same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis = chanDim))
    model.add(Conv2D(64, (3, 3), padding = "same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis = chanDim))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.4))

    # (CONV -> RELU) * 3 -> POOL layer set
    model.add(Conv2D(128, (3, 3), padding = "same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis = chanDim))
    model.add(Conv2D(128, (3, 3), padding = "same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis = chanDim))
    model.add(Conv2D(128, (3, 3), padding = "same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis = chanDim))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.4))

    # FC -> RELU layer set
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.8))

    # softmax classifier
    model.add(Dense(11, kernel_regularizer = 'l2'))
    model.add(Activation("softmax"))

    return model

In [18]:
def preprocessing():
    '''
    preprocessing(): Preprocesses images in hair dataset.
    \t:return: tuple containing features and labels for images in hair dataset
    '''

    # access hair dataset from directory
    DIRECTORY = r'new_dataset'
    CATEGORIES = ['1', '2A', '2B', '2C', '3A', '3B', '3C', '4A', '4B', '4C', 'no_hair']
    ENCODINGS = {
        '1': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        '2A': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        '2B': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        '2C': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        '3A': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        '3B': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        '3C': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        '4A': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        '4B': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        '4C': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        'no_hair': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
    }

    data = []

    # convert images to numpy arrays, and add them to data array
    for category in CATEGORIES:
        folder = os.path.join(DIRECTORY, category)

        for img in os.listdir(folder):
            img_path = os.path.join(folder, img)
            if (".DS_Store" in img_path):
                continue
            img_arr = cv2.imread(img_path)
            img_arr = cv2.resize(img_arr, (224, 224))
            encoding = ENCODINGS.get(category)
            data.append([img_arr, encoding])

    # shuffle data
    random.shuffle(data)

    X = []
    Y = []

    # separate features and labels
    for features, labels in data:
        X.append(features)
        Y.append(labels)

    X = np.array(X)
    Y = np.array(Y)

    # return all features and labels
    return (X, Y)

In [19]:
def test(model):
    '''
    test(): Implements the testing phase of the CHC model.
    \t:param model: trained CHC model
    '''

    TEST_DATASET = r'test_dataset'
    for img in os.listdir(TEST_DATASET):
        # convert image to numpy array
        img_arr = cv2.imread(img)
        img_arr = cv2.resize(img_arr, (224, 224))

        # show the image
        plt.imshow(img)
        plt.show()

        # model makes prediction
        prediction = model.predict(img.reshape(-1, 224, 224, 3))

        # print model prediction
        print("Prediction = " + str(prediction))
    
    model.save(r'v2_base')

In [61]:
def train_and_validate():
    '''
    train_and_validate(): Implements the training and validation phases of the CHC model.
    '''

    # receive features and labels after preprocessing
    tup = preprocessing()
    X = tup[0]
    Y = tup[1]
    
    # split hair dataset into training, validating, and testing datasets
    """
    x_remainder, x_test, y_remainder, y_test = train_test_split(X, Y, test_size = 0.1, random_state = 42)
    x_train, x_valid, y_train, y_valid = train_test_split(x_remainder, y_remainder, test_size = 0.1, random_state = 42)
    """
    x_train, x_valid, y_train, y_valid = train_test_split(X, Y, test_size = 0.2, random_state = 42)

    # normalize datasets
    x_train = x_train / 255.0
    x_valid = x_valid / 255.0

    # set up tensorboard
    NAME = f'hair-type-prediction-{int(time.time())}' 
    tensorboard = TensorBoard(log_dir=f'logs\\{NAME}\\')
    
    # construct the model; implement training and validation
    model = build(224, 224, 3)
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    model.summary()
    # model.fit(x_train, y_train, epochs = 50, batch_size = 20, validation_data = (x_valid, y_valid), callbacks = tensorboard)
    model.fit(x_train, y_train, epochs = 25, batch_size = 20, validation_data = (x_valid, y_valid))

    # transition to testing
    test(model)

In [62]:
train_and_validate()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_46 (Conv2D)          (None, 224, 224, 32)      896       
                                                                 
 activation_27 (Activation)  (None, 224, 224, 32)      0         
                                                                 
 batch_normalization_23 (Bat  (None, 224, 224, 32)     128       
 chNormalization)                                                
                                                                 
 max_pooling2d_39 (MaxPoolin  (None, 112, 112, 32)     0         
 g2D)                                                            
                                                                 
 dropout_24 (Dropout)        (None, 112, 112, 32)      0         
                                                                 
 conv2d_47 (Conv2D)          (None, 112, 112, 64)    

[ WARN:0@12723.829] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('test_4A.jpeg'): can't open/read file: check file path/integrity


error: OpenCV(4.6.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [ ]:
# needs to get updated
def make_prediction(img):
    # retrieve model
    model = models.load_model(r'v2_base')
    
    # model makes prediction
    prediction = model.predict(img.reshape(-1, 224, 224, 3))

    ENCODINGS = {
        '1': 0,
        '2A': 1,
        '2B': 2,
        '2C': 3,
        '3A': 4,
        '3B': 5,
        '3C': 6,
        '4A': 7,
        '4B': 8,
        '4C': 9,
        'no_hair': 10
    }

    top = (-1 * sys.maxsize) - 1
    index = -1
    for i in range(len(prediction[0])):
        if prediction[0][i] > top:
            top = prediction[0][i]
            index = i
    
    for key in ENCODINGS:
        if ENCODINGS[key] == index:
            return key

demo = gr.Interface(fn = make_prediction, inputs = gr.Image(shape=(224, 224)), outputs = gr.Label(num_top_classes = 1)).launch()

In [ ]:
def documentation():
    print(imports.__doc__)
    print(preprocessing.__doc__)
    print(build.__doc__)
    print(train_and_validate.__doc__)
    print(test.__doc__)

documentation()